In [ ]:
%matplotlib inline
import numpy as np
import scipy
import matplotlib.pyplot as plt

import vdmlab as vdm
from load_data import get_events

In [ ]:
import info.check as info

In [ ]:
events = get_events(info.event_mat)
events['label']

In [ ]:
events['cue']

In [ ]:
plt.plot(events['main_off'])
plt.show()

In [ ]:
plt.plot(events['cue'])
plt.show()

In [ ]:
plt.plot(events['house'])
plt.show()